In [2]:
import os
path_to_data = '../../datasets/Machine_translation_EN_RU/data.txt'
if not os.path.exists(path_to_data):
    print("Dataset not found locally. Downloading from github.")
    !wget https://raw.githubusercontent.com/neychev/made_nlp_course/master/datasets/Machine_translation_EN_RU/data.txt -nc
    path_to_data = './data.txt'

Dataset not found locally. Downloading from github.
--2024-03-03 12:52:25--  https://raw.githubusercontent.com/neychev/made_nlp_course/master/datasets/Machine_translation_EN_RU/data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12905334 (12M) [text/plain]
Saving to: ‘data.txt’

data.txt            100%[===================>]  12.31M  --.-KB/s    in 0.07s   

2024-03-03 12:52:25 (179 MB/s) - ‘data.txt’ saved [12905334/12905334]



In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

import torchtext
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer
from nltk.tokenize import wordpunct_tokenize

import numpy as np
import pandas as pd
import time

import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

# Preprocessing



In [4]:
MAX_SEQUENCE_LENGTH = 50
BATCH_SIZE = 128

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# ru_tokenizer = get_tokenizer('spacy', language='ru')
# en_tokenizer = get_tokenizer('spacy', language='en')

ru_tokenizer = wordpunct_tokenize
en_tokenizer = wordpunct_tokenize

In [6]:
# Step 1: Read the dataset file and parse it
def read_dataset(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    english_sentences = []
    russian_sentences = []
    for line in lines:
        english, russian = line.strip().split('\t')
        english_sentences.append(english)
        russian_sentences.append(russian)
    return english_sentences, russian_sentences

en_sentences, ru_sentences = read_dataset(path_to_data)

In [7]:
# Step 2: Tokenize the sentences
def tokenize_sentences(sentences, tokenizer):
    tokenized_sentences = [tokenizer(sentence) for sentence in sentences]
    return tokenized_sentences

ru_tokenized = tokenize_sentences(ru_sentences, ru_tokenizer)
en_tokenized = tokenize_sentences(en_sentences, en_tokenizer)

In [8]:
print(ru_tokenized[0])

['Отель', 'Cordelia', 'расположен', 'в', 'Тбилиси', ',', 'в', '3', 'минутах', 'ходьбы', 'от', 'Свято', '-', 'Троицкого', 'собора', '.']


In [9]:
# Step 3: Create vocabulary mappings
def build_vocab(tokenized_sentences):
    counter = Counter()
    for sentence in tokenized_sentences:
        counter.update(sentence)
    vocab = torchtext.vocab.vocab(counter, min_freq=4, specials=('<UNK>', '<SOS>', '<EOS>', '<PAD>'))
    vocab.set_default_index(vocab["<UNK>"])
    return vocab

ru_vocab = build_vocab(ru_tokenized)
en_vocab = build_vocab(en_tokenized)

In [10]:
SRC_VOCAB_SIZE = len(ru_vocab)
TGT_VOCAB_SIZE = len(en_vocab)
PAD_IDX = en_vocab['<PAD>']
SOS_IDX = en_vocab['<SOS>']
EOS_IDX = en_vocab['<EOS>']

In [11]:
print(len(en_vocab))
print(len(ru_vocab))

6718
8706


In [12]:
# Step 4: Transform data to torch Tensor and make train, val, test split
data = []
for (ru_sentence, en_sentence) in zip(ru_tokenized, en_tokenized):
    ru_tensor_ = torch.tensor(ru_vocab.lookup_indices(ru_sentence), dtype=torch.long)
    en_tensor_ = torch.tensor(en_vocab.lookup_indices(en_sentence), dtype=torch.long)
    data.append((ru_tensor_, en_tensor_))

train_size = int(0.8 * len(data))
val_size = int(0.15 * len(data))
test_size = len(data) - train_size - val_size
train_data, val_data, test_data = torch.utils.data.random_split(data, [train_size, val_size, test_size])

In [13]:
# Step 5: Make batches and data loaders
def generate_batch(data_batch):
    ru_batch, en_batch = [], []
    for (ru_item, en_item) in data_batch:
        ru_batch.append(torch.cat([torch.tensor([ru_vocab['<SOS>']]),
                                   ru_item,
                                   torch.tensor([ru_vocab['<EOS>']])
                                   ], dim=0))
        en_batch.append(torch.cat([torch.tensor([en_vocab['<SOS>']]),
                                   en_item,
                                   torch.tensor([en_vocab['<EOS>']])
                                   ], dim=0))
    ru_batch = pad_sequence(ru_batch, padding_value=ru_vocab['<PAD>'])
    en_batch = pad_sequence(en_batch, padding_value=en_vocab['<PAD>'])
    return ru_batch, en_batch


train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
val_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)

In [14]:
batch = next(iter(train_iter))

In [15]:
print(batch[0].shape)

torch.Size([50, 128])


# Определение модели

In [16]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * np.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * np.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = nn.Transformer(d_model=emb_size,
                                          nhead=nhead,
                                          num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers,
                                          dim_feedforward=dim_feedforward,
                                          dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [17]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [18]:
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


# Обучение

In [19]:
def train_epoch(model, optimizer, n_show = 50, train_history=None, valid_history=None):
    model.train()
    losses = 0

    epoch_history = []

    I_val = []
    val_history = []
    for i, batch in enumerate(train_iter):
        src = batch[0].to(DEVICE)
        tgt = batch[1].to(DEVICE)

        optimizer.zero_grad()

        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()

        epoch_history.append(loss.cpu().data.numpy())

        losses += loss.item()

        if (i+1) % n_show == 0:
            I_val.append(i)
            val_history.append(evaluate(model))
            model.train()

            fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))

            clear_output(True)
            ax[0].plot(epoch_history, label='train loss')
            ax[0].plot(I_val, val_history, label='validation loss')
            ax[0].set_xlabel('Batch')
            ax[0].set_title('Loss')
            ax[0].legend()
            if train_history is not None:
                ax[1].plot(train_history, label='general train history')
                ax[1].set_xlabel('Epoch')
            if valid_history is not None:
                ax[1].plot(valid_history, label='general valid history')
            plt.legend()

            plt.show()


    return losses / len(list(train_iter))


def evaluate(model):
    model.eval()
    losses = 0

    for src, tgt in val_iter:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_iter))

In [20]:
from timeit import default_timer as timer
NUM_EPOCHS = 4
n_show = 50

train_history = []
valid_history = []

In [ ]:
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer, n_show, train_history, valid_history)
    end_time = timer()
    val_loss = evaluate(transformer)

    train_history.append(train_loss)
    valid_history.append(val_loss)

    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


In [84]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, src.shape[1]).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)

        ys = torch.cat([ys, next_word.unsqueeze(0)], dim=0)
    return ys

In [85]:
from nltk.translate.bleu_score import corpus_bleu
import tqdm

In [98]:
# Translate test data and get model translations
translated_sentences = []
target_sentences = []

for batch in tqdm.tqdm(test_iter):
    transformer.eval()
    src = batch[0].to(DEVICE)
    trg = batch[1].to(DEVICE)

    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)

    with torch.no_grad():
        # Assuming your model returns token indices
        translated_indices = greedy_decode(transformer, src, src_mask, 60, SOS_IDX)
        translated_indices = translated_indices.transpose(0, 1).cpu().numpy().tolist()

        for indices in translated_indices:
            tmp = []
            for idx in indices:
                if idx == 0 or idx > 3:
                    tmp.append(en_vocab.lookup_token(idx))
                elif idx == PAD_IDX:
                    break
            translated_sentences.append(tmp)

        trg_indices = trg.transpose(0, 1).cpu().numpy().tolist()
        for indices in trg_indices:
            tmp = []
            for idx in indices:
                if idx == 0 or idx > 3:
                    tmp.append(en_vocab.lookup_token(idx))
                elif idx == PAD_IDX:
                    break
            target_sentences.append(tmp)


100%|██████████| 20/20 [00:32<00:00,  1.65s/it]


In [101]:
# Calculate BLEU score
bleu_score = corpus_bleu([[text] for text in target_sentences], translated_sentences) * 100
print("BLEU Score:", bleu_score)

BLEU Score: 13.656677608073414
